<a href="https://colab.research.google.com/github/SiMori92/reddit-NLP/blob/main/Step4_Training_Keywords_time_series_binary_v3_(post_submit_revision).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem we solve:
- Correlation between Reddit comment and Stock market performance

Content

- Pre1: load library
- Pre2: load data
  - split training set & testing set
1. Training
  1. models
    - SVM
    - Gaussian Naive Bayes
    - Random Forest
    - XGBoost
    - KNN
    - Perceptron
    - Stochastic Gradient Descent (SGD)
2. Evaluation
  1. Accuracy
  2. F1
3. Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import library

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import string

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Libraries for text preprocessing.
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

!pip install bert-tensorflow
from bert import tokenization

from wordcloud import STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 669.2 kB/s eta 0:00:00


In [ ]:
# #load traing features

X_train_binary = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CUSCS/CUSCS - final project/data/X_train_binary_Oct22-Jun23.csv", index_col=0)

X_test_binary = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CUSCS/CUSCS - final project/data/X_test_binary_Oct22-Jun23.csv", index_col=0)

Train_labels_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CUSCS/CUSCS - final project/data/Y_train_binary_Oct22-Jun23.csv", index_col=0)

Test_labels_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CUSCS/CUSCS - final project/data/Y_test_binary_Oct22-Jun23.csv", index_col=0)

X_train_binary


,Positive Score,Neutral Score,Negative Score,Volume_NQ,Positive Score_lag1,Positive Score_lag2,Neutral Score_lag1,Neutral Score_lag2,Negative Score_lag1,Negative Score_lag2,Volume_NQ_lag1,Volume_NQ_lag2
date,,,,,,,,,,,,
2022-07-13,5024.359219,5678.274264,4083.893385,4109390000,3776.735609,3776.076531,4296.107199,4295.832374,3074.200317,3074.086584,4.288924e+09,4.290283e+09
2022-07-14,4844.542607,5532.941506,3947.415250,4199690000,5024.359219,3776.076531,5678.274264,4295.832374,4083.893385,3074.086584,4.109390e+09,4.290283e+09
2022-07-15,5196.744694,5936.066150,4100.179924,4143800000,4844.542607,5024.359219,5532.941506,5678.274264,3947.415250,4083.893385,4.199690e+09,4.109390e+09
2022-07-19,4271.226150,4895.476156,3544.895066,4041070000,5196.744694,4844.542607,5936.066150,5532.941506,4100.179924,3947.415250,4.143800e+09,4.199690e+09
2022-07-20,5478.581053,6209.195254,4376.094286,4185300000,4271.226150,5196.744694,4895.476156,5936.066150,3544.895066,4100.179924,4.041070e+09,4.143800e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-25,2804.523721,3157.972777,2268.912828,3978640000,2104.656566,2198.441592,2392.546549,2522.740930,1688.785950,1801.110241,3.290940e+09,3.611750e+09
2023-04-26,3928.897786,4420.336524,3169.792346,3837030000,2804.523721,2104.656566,3157.972777,2392.546549,2268.912828,1688.785950,3.978640e+09,3.290940e+09
2023-04-27,3117.071916,3533.111601,2482.705836,3750550000,3928.897786,2804.523721,4420.336524,3157.972777,3169.792346,2268.912828,3.837030e+09,3.978640e+09


In [ ]:
# #load traing labels

Y_train_binary = Train_labels_df['Up']
Y_train_binary

Y_test_binary = Test_labels_df['Up']
Y_test_binary

date
2023-05-02    0
2023-05-03    0
2023-05-04    0
2023-05-05    1
2023-05-08    1
2023-05-09    0
2023-05-10    0
2023-05-11    0
2023-05-12    0
2023-05-15    1
2023-05-16    0
2023-05-17    1
2023-05-18    1
2023-05-19    0
2023-05-22    1
2023-05-23    0
2023-05-24    0
2023-05-25    0
2023-05-26    1
2023-05-30    0
2023-05-31    0
2023-06-01    1
2023-06-02    1
2023-06-06    1
2023-06-07    0
2023-06-08    1
2023-06-09    0
2023-06-12    1
2023-06-13    1
2023-06-14    1
2023-06-15    1
2023-06-16    0
2023-06-20    0
2023-06-21    0
2023-06-22    1
2023-06-23    0
2023-06-26    0
2023-06-27    1
2023-06-28    1
2023-06-29    1
2023-06-30    1
Name: Up, dtype: int64

## 4. Training

In [ ]:
# training models

# LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

# SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# XGBoost
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Perceptron
from sklearn.linear_model import Perceptron

# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

In [ ]:
# Deep learning models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1337

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# Gaussian Naive Bayes

nb_clf = GaussianNB()
nb_clf.fit(X_train_binary.values, Y_train_binary.values)

GaussianNB()

In [ ]:
# SVM
# SVM doesn't support multiclass classification natively

clf_svm = SVC(kernel='linear', C=1, max_iter=500)
clf_svm.fit(X_train_binary.values, Y_train_binary.values)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=1, kernel='linear', max_iter=500)

In [ ]:
# Random Forest

# the parameters meaning
#  n_estimators 随机森林中树的个数，即学习器的个数。
# n_jobs 并行使用的进程数，默认1个，如果设置为-1，该值为总的核数
# max_depth 树的最大深度
# random_state =42 just a common practice

classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=15,
                                       n_estimators=500)

# Random Forest trained with NQ

classifier_rf.fit(X_train_binary.values, Y_train_binary.values)

RandomForestClassifier(max_depth=15, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [ ]:
# Random Forest with XGB

gbm_param_grid = {
    'n_estimators': range(8, 300),
    'max_depth': range(6, 20),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}
gbm = XGBClassifier(n_estimators=300)
xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid,
                                    estimator = gbm, scoring = "accuracy",
                                    verbose = 1, n_iter = 50, cv = 4)
# Random Forest with XGB trained with NQ

xgb_random.fit(X_train_binary.values, Y_train_binary.values)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=300, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=50,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1],
                                        'learning_rate': [0.4, 0.45, 0.5, 0.55,
                                                          0.6],
                                        'max_depth': range(6, 20),
                                        'n_estimators': range(8, 300)},
                   scoring='accuracy', verbose=1)

In [ ]:
# KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors = 10)
knn_clf.fit(X_train_binary.values, Y_train_binary.values)

KNeighborsClassifier(n_neighbors=10)

In [ ]:
# Perceptron

per_clf = Perceptron(max_iter=10, tol=None)
per_clf.fit(X_train_binary.values, Y_train_binary.values)

Perceptron(max_iter=10, tol=None)

In [ ]:
# Stochastic Gradient Descent (SGD)

sgd_clf = SGDClassifier(max_iter=10, tol=None)
sgd_clf.fit(X_train_binary.values, Y_train_binary.values)

SGDClassifier(max_iter=10, tol=None)

In [42]:
# Deep learning models

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_binary)

# Convert target labels to one-hot encoded format
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train_binary)

# Determine the number of unique classes
num_classes = len(label_encoder.classes_)

# Create a sequential model
deep_clf = Sequential([
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')  # Use sigmoid activation for binary classification
])

# Compile the model
deep_clf.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with a specified number of epochs
deep_clf.fit(X_train_scaled, Y_train_encoded, epochs=1000, batch_size=30)


Epoch 1/1000
5/5 [==============================] - 1s 5ms/step - loss: 0.7996 - accuracy: 0.4867
Epoch 2/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7819 - accuracy: 0.5000
Epoch 3/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7709 - accuracy: 0.5067
Epoch 4/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7622 - accuracy: 0.5067
Epoch 5/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7542 - accuracy: 0.5200
Epoch 6/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7479 - accuracy: 0.5200
Epoch 7/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7417 - accuracy: 0.5133
Epoch 8/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7362 - accuracy: 0.5200
Epoch 9/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.7310 - accuracy: 0.5267
Epoch 10/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7258 - accuracy: 0.5200
Epoch 11/1000
5/5 [

## 5. Evaluation

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Gaussian Naive Bayes
y_predict_bayes = nb_clf.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_bayes))

              precision    recall  f1-score   support

           0       0.55      0.81      0.65        21
           1       0.60      0.30      0.40        20

    accuracy                           0.56        41
   macro avg       0.57      0.55      0.53        41
weighted avg       0.57      0.56      0.53        41



In [ ]:
# SVM

y_predict_svm = clf_svm.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_svm))

              precision    recall  f1-score   support

           0       0.67      0.10      0.17        21
           1       0.50      0.95      0.66        20

    accuracy                           0.51        41
   macro avg       0.58      0.52      0.41        41
weighted avg       0.59      0.51      0.40        41



In [ ]:
# Random forest
y_predict_rf = classifier_rf.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_rf))

              precision    recall  f1-score   support

           0       0.33      0.14      0.20        21
           1       0.44      0.70      0.54        20

    accuracy                           0.41        41
   macro avg       0.39      0.42      0.37        41
weighted avg       0.38      0.41      0.37        41



In [ ]:
# Random Forest with XGB

y_predict_xgb = xgb_random.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_xgb))

              precision    recall  f1-score   support

           0       0.44      0.33      0.38        21
           1       0.44      0.55      0.49        20

    accuracy                           0.44        41
   macro avg       0.44      0.44      0.43        41
weighted avg       0.44      0.44      0.43        41



In [ ]:
# KNN
y_predict_KNN = knn_clf.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_KNN))

              precision    recall  f1-score   support

           0       0.63      0.57      0.60        21
           1       0.59      0.65      0.62        20

    accuracy                           0.61        41
   macro avg       0.61      0.61      0.61        41
weighted avg       0.61      0.61      0.61        41



In [ ]:
# Perceptron

y_predict_per = per_clf.predict(X_test_binary.values)
print(classification_report(Y_test_binary,y_predict_per))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68        21
           1       0.00      0.00      0.00        20

    accuracy                           0.51        41
   macro avg       0.26      0.50      0.34        41
weighted avg       0.26      0.51      0.35        41



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Stochastic Gradient Descent (SGD)

y_predict_sgd = sgd_clf.predict(X_test_binary.values)

print(classification_report(Y_test_binary,y_predict_sgd))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.49      1.00      0.66        20

    accuracy                           0.49        41
   macro avg       0.24      0.50      0.33        41
weighted avg       0.24      0.49      0.32        41



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
#Deep learning

# Predict the probabilities for each class
y_predict_deep_probabilities = deep_clf.predict(X_test_binary.values)
y_predict_deep_probabilities = y_predict_deep_probabilities.flatten()  # Flatten the probabilities array

# Convert probabilities to class labels
y_predict_deep_labels = np.where(y_predict_deep_probabilities >= 0.5, 1, 0)

# Print the classification report
print(classification_report(Y_test_binary, y_predict_deep_labels))

2/2 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

           0       1.00      0.14      0.25        21
           1       0.53      1.00      0.69        20

    accuracy                           0.56        41
   macro avg       0.76      0.57      0.47        41
weighted avg       0.77      0.56      0.46        41



## 6. Analysis

- Accuracy: all models below or equal 0.61
-------------
- Precision of down (0) :1 Deep learning vs 0 SGD
- Precision of up (1) : 0.6 Bayes vs 0 Perceptron
-------------
- F1-score ( weighted average of precision and recall) of down (0): 0.68 Perceptron vs 0 SGD
- F1-score ( weighted average of precision and recall) of up (1): 0.69 Deep learning vs 0 Perceptron





In [ ]:
# if doing AND to Random forest vs KNN

y_predict_combined_and = np.logical_and(y_predict_rf, y_predict_KNN).astype(int)
y_predict_combined_and

print(classification_report(NQ_test,y_predict_combined_and))

In [ ]:
# if doing OR to Random forest vs KNN

y_predict_combined_or = np.logical_or(y_predict_rf, y_predict_KNN).astype(int)
y_predict_combined_or

print(classification_report(NQ_test,y_predict_combined_or))